In [1]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

[u'\n', u'!', u' ', u"'", u')', u'(', u',', u'.', u';', u':', u'?', u'a', u'c', u'b', u'e', u'd', u'g', u'f', u'i', u'h', u'k', u'j', u'm', u'l', u'o', u'n', u'p', u's', u'r', u'u', u't', u'w', u'y', u'z', u'a', u'c', u'b', u'e', u'd', u'g', u'f', u'i', u'h', u'k', u'j', u'm', u'l', u'o', u'n', u'q', u'p', u's', u'r', u'u', u't', u'w', u'v', u'y', u'x', u'z']


In [3]:
vocab_size = 1651
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print len(d)
x, y = d[10:20], d[11:21]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

[u'we', u'have', u'waited', u'for', u'this', u'moment', u'for', u'a', u'long', u'time', u'.', u'as', u'a', u'small', u'startup', u'with', u'resource', u'constraints', u'but', u'ambitious', u'goals', u'to', u'reach', u'the', u'pinnacle', u'of', u'what\u2019s', u'possible', u'in', u'artificial', u'intelligence', u'research', u',', u'we', u'think', u'we', u'have', u'come', u'a', u'long']
1651
8393
[1, 16, 5, 261, 62, 12, 1046, 1478, 43, 248]
[16, 5, 261, 62, 12, 1046, 1478, 43, 248, 1632]


In [4]:
# hyper-parameters
input_size = output_size = vocab_size
hidden_layer = 300
inp_out_size = vocab_size
learning_rate = 0.1
num_steps = 20

In [5]:
tf.reset_default_graph()
initializer = tf.random_normal_initializer(mean=0, stddev=0.001, dtype=tf.float32)
Wxh = tf.get_variable('Wxh', shape=[input_size, hidden_layer], initializer=initializer)
Whh = tf.get_variable('Whh', shape=[hidden_layer, hidden_layer], initializer=initializer)
Why = tf.get_variable('Why',shape=[hidden_layer, output_size], initializer=initializer)
by = tf.get_variable('by', shape=[output_size], initializer=initializer)
# weights associated with update gate
Wxz = tf.get_variable('Wxz', shape=[input_size, hidden_layer], initializer=initializer)
Whz = tf.get_variable('Whz', shape=[hidden_layer, hidden_layer], initializer=initializer)
# weights associated with the reset gate
Wxr = tf.get_variable('Wxr', shape=[input_size, hidden_layer], initializer=initializer)
Whr = tf.get_variable('Whr', shape=[hidden_layer, hidden_layer], initializer=initializer)

In [6]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, Wxz) + tf.matmul(p, Whz))    # update gate
    r = tf.nn.sigmoid(tf.matmul(i, Wxr) + tf.matmul(p, Whr))    # reset gate
    h_ = tf.nn.tanh(tf.matmul(i, Wxh) + tf.matmul(tf.mul(p, r), Whh))
    h = tf.nn.tanh(tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p))
    return tf.reshape(h, [hidden_layer])

In [7]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.nn.dropout(tf.scan(recurrence, a, initializer=initial), keep_prob=0.8)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -4, 4), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

# optimize_op = optimizer.minimize(loss)

In [12]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print ' '.join(gen)

In [9]:
init = tf.initialize_all_variables()
saver = tf.train.Saver()
sess = tf.Session()
ix = 0

In [10]:
# if already trained previously, just restore
to_restore = False

if to_restore:
    saver.restore(sess, 'model.ckpt')
else:
    sess.run(init)

In [13]:
iterations = 10000
h = np.zeros(hidden_layer)
for i in range(iterations):
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
        print('one epoch complete')
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, o, _ = sess.run([loss, outputs, optimize_op], {a: x, b: y, initial: h})
    if i % 500 == 0:
        print('iteration {0}, loss = {1}'.format(i, l))
        generate(sess, 50)
    ix += num_steps

iteration 0, loss = 146.211547852
on a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
one epoch complete
iteration 500, loss = 117.030181885
my turing . we have a lot of our of our . we have have to have a lot of the system of the time . we have to build a lot of our . we have a lot of our of our . we have a lot of our .
one epoch complete
iteration 1000, loss = 73.7587127686
product . we have it to build the market . we are a lot to build a year . with technology and robotics . we have it was a great time . we are going to build a year . we are going to be our products and robotics . we
one epoch complete
iteration 1500, loss = 83.6778793335
this is a lot of the world is that we are no longer no longer way to build product and become the market . we are no longer to product and it has become a lot of the new market . and we are no longer no longer system to build
one epoch complete
iteration 2000, loss = 55.8870697021
us . he was pret

In [ ]:
# saving model
# saver.save(sess, 'model.ckpt')

In [14]:
generate(sess, 100)

much traction which itself required marketing dollars.we were stuck in a vicious loop , to be able to pursue his robotics and robotics passion before that time and that should be part of your product market fit , an industry has , by a month ago , this is an one of the human computer interaction ) . image is very a time . we have a great set to new yourself focused and technology innovation . in the very half of our product or on . and robotics and we could do any path we wanted to and that helped


In [ ]:
# sess.run(Whh)